In [8]:
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import gensim.downloader as gensim
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
gensim_model = gensim.load("word2vec-ruscorpora-300")
import re

In [9]:
data = pd.read_csv('banks.csv', sep='\t')
data = data.drop('idx', axis =1)
data['Score'] = data['Score'].replace({'Negative': 0, 'Positive': 1})

In [10]:
X,y = data['Score'].values, data['Text'].values
y_train,y_test,x_train,x_test = train_test_split(X,y,test_size=0.33, random_state=69)
print(f'shape of train data is {x_train.shape}')
print(f'shape of test data is {x_test.shape}')

shape of train data is (9379,)
shape of test data is (4620,)


In [11]:
tokenizer = RegexpTokenizer("[\w']+")

def get_vector(x):
    try:
        return gensim_model.get_vector(x)
    except:
        return np.zeros(shape=300)

def preprocess(data):
    stop = set(stopwords.words('russian'))
    x_train = []
    for text in data:
        text = re.sub(r'http\S+', '', text)
        sentence_tokens = [i for i in tokenizer.tokenize(text.lower()) if i not in stop]
        lemmatized = []
        for word in sentence_tokens:
            lemma = morph.parse(word)[0].normal_form
            lemma = lemma + '_'
            lemma = lemma + str(morph.parse(word)[0][1])[0:4]
            lemmatized.append(lemma)
        x_train.append(lemmatized)
    # to vectors:
    review_vectors = []
    for reviews in x_train:
        vectors = []
        for token in reviews:
            vectors.append(get_vector(token))
        review_vectors.append(np.sum(vectors, axis=0))
        
    return review_vectors



In [12]:
x_train = preprocess(x_train)

In [13]:
x_test = preprocess(x_test)

In [31]:
a = torch.Tensor(x_train)
a.size(), a

(torch.Size([9379, 300]),
 tensor([[ 2.4571, -1.0436, -2.9950,  ..., -0.2068, -2.5636, -1.0552],
         [ 0.3721, -0.1147, -0.5656,  ..., -0.1386, -0.4819, -0.1435],
         [ 0.8455, -0.2086, -0.1971,  ...,  0.3741, -0.6512, -0.0865],
         ...,
         [ 1.1214,  0.8105, -0.3891,  ...,  0.0730, -2.5163, -0.7254],
         [ 1.2723, -0.1648, -0.6650,  ...,  0.7105, -1.3658, -1.1616],
         [ 2.2741,  0.6004, -0.5608,  ...,  0.3783, -3.0146, -1.2248]]))

## RNN Model

In [15]:
import torch.nn.functional as F
import string
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
import torch 
import torch.optim as optim
device = torch.device("cuda")

In [16]:
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


In [9]:
vocab_size = len(x_train)
embedding_dim = x_train[0].shape
x_train[0].shape

(300,)

In [22]:
train_data = TensorDataset(torch.from_numpy(np.asarray(x_train)), torch.from_numpy(np.asarray(y_train)))
valid_data = TensorDataset(torch.from_numpy(np.asarray(x_test)), torch.from_numpy(np.asarray(y_test)))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [114]:
h0 = torch.randn(2, 3, 20)
h0.size()

torch.Size([2, 3, 20])

In [115]:

class RNN(nn.Module):
    def __init__(self, input_dim, input_size, hidden_dim, output_dim): # input_size = [50, 300], input_dim = 2, hidden_dim = [50, 256], output_dim = 1
        super().__init__()
        
        #self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(input_size, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = [300, 50, 2]
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        #assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [117]:
INPUT_DIM = len(x_train)
EMBEDDING_DIM = [50, 300]
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got list"

In [87]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 143,105 trainable parameters


In [89]:


optimizer = optim.SGD(model.parameters(), lr=1e-3)

criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [90]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [104]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [103]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [101]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [102]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

AttributeError: 'list' object has no attribute 'dim'

In [79]:
class SentimentRNN(nn.Module):
    def __init__(self,no_layers,hidden_dim,embedding_dim,drop_prob=0.5):
        super(SentimentRNN,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
 
        self.no_layers = no_layers

        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
    
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = torch.Size([50, 300])
        # embeddings and lstm_out
        lstm_out, hidden = self.lstm(embedding_dim, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
        
        
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

In [75]:
no_layers = 2
#vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 300
output_dim = 1
hidden_dim = 256


model = SentimentRNN(no_layers,hidden_dim,embedding_dim,drop_prob=0.5)

#moving to gpu
model.to(device)

print(model)

SentimentRNN(
  (lstm): LSTM(300, 256, num_layers=2, batch_first=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [76]:
# loss and optimization functions
lr=0.001

criterion = nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [77]:
clip = 5
epochs = 5 
valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state 
    h = model.init_hidden(batch_size)
    for inputs, labels in train_loader:
        
        inputs, labels = inputs.to(device), labels.to(device)   
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        
        model.zero_grad()
        output,h = model(inputs,h)
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        train_losses.append(loss.item())
        # calculating accuracy
        accuracy = acc(output,labels)
        train_acc += accuracy
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
 
    
        
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for inputs, labels in valid_loader:
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.to(device), labels.to(device)

            output, val_h = model(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())
            
            accuracy = acc(output,labels)
            val_acc += accuracy
            
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader.dataset)
    epoch_val_acc = val_acc/len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), '../working/state_dict.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')

AttributeError: 'numpy.ndarray' object has no attribute 'dim'

In [17]:
input_size = 300
sequence_length = 1
num_layers=2
hidden_size = 256
num_classes = 1
lr = 0.001
batch_size = 64
num_epochs = 2

In [18]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first = True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)
        out = self.fc(out)

        return out

In [19]:

train_data = TensorDataset(torch.from_numpy(np.asarray(x_train)), torch.from_numpy(np.asarray(y_train)))
test_data = TensorDataset(torch.from_numpy(np.asarray(x_test)), torch.from_numpy(np.asarray(y_test)))


# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [34]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)
sample_x, sample_y = next(dataiter)

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)
print('Sample input size: ', sample_y.size())

sample_x.dim()

Sample input size:  torch.Size([50, 300])
Sample input: 
 tensor([[ 2.1614, -0.6034, -1.3480,  ...,  2.2637, -3.5038, -2.0732],
        [ 0.5521,  0.1128, -0.2699,  ..., -0.3646, -1.7077, -0.2140],
        [ 2.5691, -0.7184, -0.6125,  ...,  0.9771, -1.1765, -0.6486],
        ...,
        [ 3.6003,  0.7328, -3.1022,  ..., -0.1816, -7.3811, -2.8741],
        [ 0.6923, -0.3605, -0.0879,  ...,  0.6766, -0.9195,  0.2370],
        [ 1.8872, -1.0936, -1.8999,  ...,  1.0703, -3.6406, -1.0661]],
       dtype=torch.float64)
Sample input: 
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 0])
Sample input size:  torch.Size([50])


2

In [22]:
rnn_model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=lr)


c:\Users\Joe\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [44]:
for epoch in range(num_epochs):
    for batch_idx, (data,targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device).squeeze(0)

        #data = data.reshape(data.shape[0], -1)

        scores = rnn_model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

RuntimeError: For unbatched 2-D input, hx should also be 2-D but got 3-D tensor

In [46]:
for epoch in range(num_epochs):
    for batch_idx, (data,targets) in enumerate(train_loader):
        print(batch_idx, '| ', data, "| ", targets)

0 |  tensor([[ 3.7603,  0.5532, -6.5908,  ...,  0.1312, -4.1758, -2.7587],
        [ 0.4526,  0.8633, -0.7577,  ...,  0.8335, -2.3223, -0.8078],
        [ 0.7302,  0.6127, -0.5947,  ...,  1.3348, -1.5056, -0.1394],
        ...,
        [ 0.7429,  0.0330, -0.2442,  ...,  0.3469, -2.0580, -0.0645],
        [ 1.0556,  1.4936,  0.1587,  ...,  0.0390, -2.1339, -0.6286],
        [ 4.9466, -0.5245, -3.4161,  ..., -1.3799, -4.5506, -2.6413]],
       dtype=torch.float64) |  tensor([0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
        0, 0])
1 |  tensor([[ 3.1815, -0.1479, -4.8089,  ..., -0.8743, -6.0566, -2.4516],
        [ 6.1200,  0.2860, -3.2466,  ..., -0.7800, -5.2232, -2.8918],
        [ 3.7348, -0.1577, -3.1236,  ..., -0.7468, -4.7248, -3.0424],
        ...,
        [ 0.3626, -0.6423, -0.6039,  ...,  0.3827, -1.6843, -0.2225],
        [ 3.2487, -0.4196, -1.1478,  ...,  0.6529, -4.427

tensor([[ 2.1276,  0.1408, -1.1790,  ...,  1.4921, -2.1904, -0.7158],
        [ 2.9361,  0.4140, -1.7365,  ...,  0.3878, -4.1036,  0.1454],
        [ 1.5207, -0.3766, -0.2797,  ...,  0.2572, -1.1037, -1.0646],
        ...,
        [ 0.6401, -0.0888, -0.1221,  ..., -0.1459, -1.4111, -0.4666],
        [ 2.7224,  0.9771, -1.9219,  ...,  2.3758, -4.2612, -0.7567],
        [ 3.2722, -0.3767, -1.1477,  ...,  0.8068, -3.3480, -1.3734]],
       dtype=torch.float64) |  tensor([0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 1])
10 |  tensor([[ 3.4642e+00, -9.5931e-01, -1.9003e+00,  ...,  6.5463e-01,
         -3.7233e+00, -1.5880e+00],
        [ 1.8909e+00, -2.0891e-01, -1.3121e+00,  ...,  4.0953e-01,
         -2.3682e+00, -9.7741e-02],
        [ 3.9419e+00, -1.2055e+00, -2.4904e+00,  ..., -1.2405e+00,
         -7.8881e+00, -2.0468e+00],
        ...,
        [ 2.5030e+00,  2.5145e-0